# watsonx.data上のベクトル・データベース Milvusを見てみよう

2025/04/18 [IBM Database Dojo 2025 watsonx.data上のベクトル・データベース Milvusを見てみよう](https://ibm-developer.connpass.com/event/351549/)のデモで使用したnotebookです。

デモでは「見てみる」ことが目的のため、プログラミングの細い説明はしません。


# 前準備

必要に応じてライブラリーをインストールしてください

In [ ]:
# !pip install -U pymilvus
# !pip install -Uq 'langchain-ibm>=0.3.1'
# !pip install -Uq 'ibm-watsonx-ai>=1.1.15'

# 2. ベクトル・データを見てみよう

##  文章のベクトル化

ここではIBMが提供するGraniteのgranite-embedding-278m-multilingualを使います(次元数768)

https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models-embed.html?context=wx&locale=ja#granite-278m-multilingual

(watsonx.dataが必要です)

### Embeddingモデル取得

#### watsonx.aiのapikey情報とProject idを設定

apikey をセットしてください。

IBM Cloud環境でのAPIKEYの取得方法は[こちら](https://qiita.com/nishikyon/items/fa8b951267efbe911fe7)を参考にしてください。

In [ ]:
import getpass
watsonx_ai_apikey = getpass.getpass("watsonx.aiのapikeyを入れてEnter Keyを押してください:")

watsonx.ai Studioで実行する場合は、このノートブックが実行されるプロジェクトからProject idを取得します。 

watsonx.ai Studio以外で実行する場合は、Project idを入力してください。<br>
**Hint**: project_id はプロジェクトを表示し、管理タブから project_id を取得可能です.

In [ ]:
import os
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("watsonx.aiのProject idを入力してください (入力後enter): ")

watsonx.aiのAuthentication用のエンドポイントのURLの設定

Waston Machine Learningのインスタンスを作成したリージョンで決まります。<br>
 https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication

- Dallas: https://us-south.ml.cloud.ibm.com
- London: https://eu-gb.ml.cloud.ibm.com
- Frankfurt: https://eu-de.ml.cloud.ibm.com
Tokyo: https://jp-tok.ml.cloud.ibm.com- 

In [ ]:
watsonx_url = "https://us-south.ml.cloud.ibm.com" #watsonx.aiのAuthentication用のエンドポイントのURL

#### watsonx.aiのEmbeddingモデル取得

In [ ]:
# watsonx.aiのEmbeddingモデル取得
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 512,
    
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

embeddings = WatsonxEmbeddings(
    model_id="ibm/granite-embedding-278m-multilingual",
    url=watsonx_url,
    apikey=watsonx_ai_apikey,
    project_id=project_id
    )

### "犬を飼っています"をベクトル化

In [ ]:
vectordata_dog = embeddings.embed_query("犬を飼っています")

print(len(vectordata_dog)) # 配列の数　= 次元数
# print(vectordata_dog[0:5]) # 最初の5個
print(vectordata_dog) # 全部

### "猫を飼っています"をベクトル化

In [ ]:
vectordata_cat = embeddings.embed_query("猫を飼っています")

print(len(vectordata_cat)) # 配列の数　= 次元数
print(vectordata_cat[0:5]) # 最初の5個
# print(vectordata_cat) #　 全部

### "大根を買っています"をベクトル化

In [ ]:
vectordata_daikon = embeddings.embed_query("大根を買っています")

print(len(vectordata_daikon)) # 配列の数　= 次元数
print(vectordata_daikon[0:5]) # 最初の5個
# print(vectordata_daikon) # 全部

### "何かペットを飼っていますか"をベクトル化

In [ ]:
vectordata_pet_question = embeddings.embed_query("何かペットを飼っていますか")
print(len(vectordata_pet_question)) # 配列の数　= 次元数
# print(vectordata_pet_question[0:5]) # 最初の5個
print(vectordata_pet_question) # 全部

### "大谷翔平の今季第1号ホームランは、2025年3月19日に東京ドームで行われたカブスとの試合でのホームランでした"をベクトル化

In [ ]:
vectordata_ohtani = embeddings.embed_query("大谷翔平の今季第1号ホームランは、2025年3月19日に東京ドームで行われたカブスとの試合でのホームランでした")

print(len(vectordata_ohtani)) # 配列の数　= 次元数
print(vectordata_ohtani[0:5]) # 最初の5個
# print(vectordata_daikon) # 全部

## 類似度を出してみる

### 以下はベクトルデータからコサイン類似度を出す関数です
(Milvus関係ない行列の計算です)

1に近いほど似ています

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 

def get_cossim_from_vector(v1,v2):
    array_1 = np.array([v1]) 
    array_2 = np.array([v2]) 
    cos_sim = cosine_similarity(array_1 , array_2) 
    print(cos_sim) 

### 類似度: "犬を飼っています"と"猫を飼っています"

In [ ]:
get_cossim_from_vector(vectordata_dog, vectordata_cat)

### 類似度: "犬を飼っています"と"大根を買っています"

In [ ]:
get_cossim_from_vector(vectordata_dog, vectordata_daikon)

### 類似度: "犬を飼っています"と"何かペットを飼っていますか"

In [ ]:
get_cossim_from_vector(vectordata_dog, vectordata_pet_question)

### 類似度: "猫を飼っています"と"何かペットを飼っていますか"

In [ ]:
get_cossim_from_vector(vectordata_cat, vectordata_pet_question)

### 類似度: "大根を買っています"と"何かペットを飼っていますか"

In [ ]:
get_cossim_from_vector(vectordata_daikon, vectordata_pet_question)

### 類似度: "大谷翔平の今季第1号ホームランは、2025年3月19日に東京ドームで行われたカブスとの試合でのホームランでした"と　"何かペットを飼っていますか"¶

In [ ]:
get_cossim_from_vector(vectordata_ohtani, vectordata_pet_question)

### ベクトル・データーベース　 -- [ベクトル検索](https://www.ibm.com/jp-ja/think/topics/vector-search)ができるデータベース
- "犬を飼っています"
- "猫を飼っています"
- "大根を買っています"
- "大谷翔平の今季第1号ホームランは、2025年3月19日に東京ドームで行われたカブスとの試合でのホームランでした。"

この中から、"何かペットを飼っていますか"の回答を見つけるなら、類似度の高い文章が選択すればよいはずです。

ただ全部比べるのは面倒ですね　👎

- "犬を飼っています"
- "猫を飼っています"
- "大根を買っています"
- "大谷翔平の今季第1号ホームランは、2025年3月19日に東京ドームで行われたカブスとの試合でのホームランでした。"
  
という4つのベクトル化したデータがDBに入っていて、<br>
これらから<br>
"何かペットを飼っていますか"のベクトル化したデータ<br>
に類似度が高いデータを検索できるのが<br>
**ベクトル・データーベースです**


# 3. watsonx.data Milvusにデータを入れ類似検索を見てみよう 

以下の4つの文章のベクトルデータをwatsonx.data Milvusに入れて、類似検索してみます
- "犬を飼っています"
- "猫を飼っています"
- "大根を買っています"
- "大谷翔平の今季第1号ホームランは、2025年3月19日に東京ドームで行われたカブスとの試合でのホームランでした"

## watsonx.data Milvusへの接続情報の設定

watsonx.data Milvusへの接続情報を取得し、設定します

参考: 
- Milvus接続情報の詳細取得手順は[こちら](https://github.com/IBM/japan-technology/tree/main/techxchange/2024-watsonx-handson-1/watsonx_data_get_milvus_info.md) を参照してください。
- my_connection_argsに設定するkeyの詳細は[pymilvusのMilvusClientのパラメータ](https://milvus.io/api-reference/pymilvus/v2.5.x/MilvusClient/Client/MilvusClient.md)を参照してください。



In [ ]:
milvus_host=input("milvus GRPC ホストを入れてEnter Keyを押してください: ")
milvus_port=input("milvus GRPC ポートを入れてEnter Keyを押してください: ")

In [ ]:
milvus_apikey = getpass.getpass("apikeyを入れてEnter Keyを押してください:")

In [ ]:
# Milvus接続情報パラメータののセット
my_connection_args ={
 'uri': f'https://{milvus_host}:{milvus_port}', 
 'token': f'ibmlhapikey:{milvus_apikey}'
}

## コレクションの作成

###  まずはwatsonx.dataのMilvusに接続します

接続情報の取得方法:[watsonx.data: Milvus接続情報の取得](https://qiita.com/nishikyon/items/e25d426cf8990991a159)

参考:[MilvusClient](https://milvus.io/api-reference/pymilvus/v2.4.x/MilvusClient/Client/MilvusClient.md)

In [ ]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri=my_connection_args['uri'],
    token=my_connection_args['token']
)

### コレクション(≒RDBでいうテーブル) のスキーマを設定します

参考: https://milvus.io/docs/ja/create-collection.md#Create-Schema

以下のように設定します

| Field        | Type              |Primary | 
|--------------|-------------------|----------|
| id           | Int64             | Yes        |
| text         | VarChar(512)      | --       |
| vector  | FloatVector(768) | --      | --      | 


In [ ]:
# Create schema
schema = MilvusClient.create_schema(
    auto_id=False
)

# 3.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=512)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=768)

### インデックスを設定します

参考: 
- https://milvus.io/docs/ja/create-collection.md#Optional-Set-Index-Parameters
- [add_index()](https://milvus.io/api-reference/pymilvus/v2.4.x/MilvusClient/Management/add_index.md#addindex)
  

以下のように設定します

| Field        | Type              |  Index Name   | Index Type  | Index Parameters       |
|--------------|-------------------|--------------|-------------|------------------------|
| id           | Int64             | --           | --   | --                     |
| text         | VarChar(512)      |  --           |--           | --                     |
| vector  | FloatVector(768) |  vector_data  | HNSW   | metric_type: COSINE    |

In [ ]:
# Prepare index parameters
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="vector", 
    index_type="HNSW",
    metric_type="COSINE"
)

### コレクションを作成します

参考: 
- https://milvus.io/docs/ja/create-collection.md#Create-a-Collection
- [create_collection()](https://milvus.io/api-reference/pymilvus/v2.4.x/MilvusClient/Collections/create_collection.md)

In [ ]:
collection_name="dojo_pet_data"

if client.has_collection(collection_name=collection_name):
    client.drop_collection(collection_name=collection_name)

client.create_collection(
    collection_name=collection_name,
    schema=schema,
    index_params=index_params
)

res = client.get_load_state(
    collection_name=collection_name
)

print(res)

## コレクションをAttuで確認
参考:
- [Attu](https://github.com/zilliztech/attu)
- Attu の導入方法: [Milvusのグラフィカルな管理ツール AttuをMacに導入してみた](https://qiita.com/nishikyon/items/addf2689812a2d9eb8e3)

## データの挿入

#### データをINSERTします

参考: https://milvus.io/docs/ja/insert-update-delete.md#Insert-Entities-into-a-Collection

In [ ]:
data=[
    {"id": 1, "text":"犬を飼っています", "vector": vectordata_dog}, 
    {"id": 2, "text":"猫を飼っています", "vector": vectordata_cat},
    {"id": 3, "text":"大根を買っています", "vector": vectordata_daikon},
    {"id": 4, "text":"大谷翔平の今季第1号ホームランは、2025年3月19日に東京ドームで行われたカブスとの試合でのホームランでした。", "vector": vectordata_ohtani}
]

In [ ]:
res = client.insert(
    collection_name="dojo_pet_data",
    data=data
)

print(res)

## データがどのようにはいったかAttuで確認 
参考:
- [Attu](https://github.com/zilliztech/attu)
- Attu の導入方法: [Milvusのグラフィカルな管理ツール AttuをMacに導入してみた](https://qiita.com/nishikyon/items/addf2689812a2d9eb8e3)

## 類似検索 



### "何かペットを飼っていますか"のベクトル・データで類似検索

In [ ]:
search_params = {
    "metric_type": "COSINE",
    "params": {}
}

# Search with limit
search_str="何かペットを飼っていますか"
res = client.search(
    collection_name="dojo_pet_data",
    data= [embeddings.embed_query(search_str)],
    # limit=3,
    output_fields=["text"],
    search_params=search_params
)

# 結果のPrint
for hits in res:
    for hit in hits:
        print(hit)

### "大谷翔平情報教えて"のベクトル・データで類似検索

In [ ]:
search_str="大谷翔平情報教えて"
res = client.search(
    collection_name="dojo_pet_data",
    data= [embeddings.embed_query(search_str)],
    # limit=3,
    output_fields=["text"],
    search_params=search_params
)

# 結果のPrint
for hits in res:
    for hit in hits:
        print(hit)

### Demoは以上です